# EDA

## Import Libraries

In [24]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import seaborn as sns
import matplotlib.pyplot as plt
import sweetviz as sv

import model.train as train
import model.config as config
import utils as utils
import model.dataset.season_game as sg

from pandas_profiling import ProfileReport
from yellowbrick.target import FeatureCorrelation
import qgrid

## Setup

In [25]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
sns.set(style="whitegrid")
#sns.set(style='whitegrid', palette=sns.color_palette(
 #   "BuGn_r"), font_scale=1.5, color_codes=True)

#pd.options.display.max_columns = None
#pd.set_option("display.max_colwidth", None)
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"
pd.set_option("display.max_rows", 50)
pd.set_option("display.max_colwidth", 50)
pd.set_option("display.max_columns", 50)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Utils functions

In [26]:
def show_qgrid(df_):
    qgrid_widget = qgrid.show_grid(df_, show_toolbar=True , grid_options={'forceFitColumns': False, 'defaultColumnWidth': 200})
    qgrid_widget

## Load dataset

In [27]:
raw_ds = sg.load_raw_season_games_dataset()
dataset = sg.load_season_games_dataset()
df = dataset

In [28]:
columns = config.X_COLS + config.Y_CLF_COL + config.Y_REG_COL

### Validate dataset

In [ ]:
cond = (dataset.WL_HOME == 'L') & (dataset.SEASON == 2018)
dataset[cond].columns.unique()

In [ ]:
test_df = pd.DataFrame({"B": [1, 2, 3, 4, 5, np.nan], "C": [1, 2, 3, 4, 5, 6], "D": [1, np.nan, 3, 4, 5, 6]})
#test_df.apply(lambda x : x.shift(1).expanding().mean())
#test_df.shift(1).expanding(min_periods=-1).mean()
test_df.shift(1).rolling(window=5, min_periods=0).sum()
#test_df[['B', 'C']] = test_df[['B', 'C']].fillna(0)




In [ ]:
cols = [
'GAME_DATE',
'MATCHUP_HOME',
'FGM_HOME',
'FGM_MEAN_HOME',
'FGA_HOME',
'FGA_MEAN_HOME',
'FG_PCT_HOME',
'FG_PCT_MEAN_HOME',
'FG3M_HOME',
'FG3M_MEAN_HOME',
'FG3A_HOME',
'FG3A_MEAN_HOME',
'FG3_PCT_HOME',
'FG3_PCT_MEAN_HOME',
'FTM_HOME',
'FTM_MEAN_HOME',    
'FTA_HOME',
'FTA_MEAN_HOME',    
'FT_PCT_HOME',
'FT_PCT_MEAN_HOME',
'OREB_HOME',
'OREB_MEAN_HOME',
'DREB_HOME',
'DREB_MEAN_HOME',
'REB_HOME',
'REB_MEAN_HOME',
'AST_HOME',
'AST_MEAN_HOME',
'STL_HOME',
'STL_MEAN_HOME',
'BLK_HOME',
'BLK_MEAN_HOME',
'TOV_HOME',
'TOV_MEAN_HOME',
'PF_HOME',
'PF_MEAN_HOME',
'PTS_HOME',
'PTS_MEAN_HOME',
'PLUS_MINUS_HOME',
'PLUS_MINUS_MEAN_HOME'
]

In [ ]:
cond = ((df.TEAM_ABBREVIATION_HOME == 'MIL') | (df.TEAM_ABBREVIATION_AWAY == 'MIL')) \
    & (df.SEASON == 2018)
#list(df[cond].columns)
df[cond][cols]

In [ ]:
pd.set_option("display.max_columns", 200)
dataset.head()

In [ ]:
ds_columns = list(dataset.columns)
ds_columns.sort()
ds_columns

### Drop unimportant column

In [29]:
#df = df[columns]

In [30]:
df[["PLUS_MINUS_ML5_HOME", "PLUS_MINUS_ML5_AWAY"]]

,PLUS_MINUS_ML5_HOME,PLUS_MINUS_ML5_AWAY
GAME_ID,,
0020200001,-2.6,7.6
0020200002,-3.8,-3.0
0020200003,3.2,-6.4
0020200004,6.2,18.2
0020200005,-6.4,15.6
...,...,...
0021901314,-3.4,3.4
0021901315,-1.8,1.8
0021901316,-4.4,4.4


### Checking null value

In [31]:
df.isnull().sum()
#.sum()

SEASON_ID                   0
TEAM_ID_HOME                0
TEAM_ABBREVIATION_HOME      0
TEAM_NAME_HOME              0
GAME_DATE                   0
                         ... 
HOME_BLK_L10              311
HOME_TOV_L10              311
HOME_PF_L10               311
HOME_PTS_L10              311
HOME_PLUS_MINUS_L10       311
Length: 381, dtype: int64

Se observa que existen valores nulos.

In [ ]:
#df.columns
pd.set_option("display.max_rows", 600)
raw_ds[raw_ds.isnull().any(axis=1)].groupby(by=["TEAM_ID", "SEASON_ID"])[['FGA']].count()

### Unique enteries in each of the columns

In [ ]:
df.nunique()

## Insights

In [ ]:
dfi = dataset.reset_index()

### WL by season

Porcentaje de victorias por temporada del equipo que juega como casa vs equipo que juega como visitante

In [ ]:
grouped = dfi[['SEASON', 'WL_HOME', 'GAME_ID']].groupby(by=['SEASON', 'WL_HOME']).count()
grouped = grouped.reset_index(level=0).reset_index(level=0)
grouped_total = grouped[['SEASON', 'GAME_ID']].groupby(by=['SEASON']).sum()
grouped_total = grouped_total.reset_index(level=0)
plot_data = pd.merge(grouped, grouped_total, suffixes=['', '_S'], on=['SEASON', 'SEASON'])
plot_data.rename(columns={"GAME_ID": "W_COUNT", "GAME_ID_S": "GAMES_COUNT"}, inplace=True)
plot_data["TEAM"] = np.where(plot_data['WL_HOME'] == 'W', 'HOME', 'AWAY')
plot_data["WINS_PERC"] = (plot_data["W_COUNT"] / plot_data["GAMES_COUNT"]) * 100
plot_data.head()

In [ ]:
import matplotlib.ticker as mtick
import matplotlib.ticker as ticker
fig_dims = (20, 10)
fig, ax = plt.subplots(figsize=fig_dims, sharex=True, sharey=True)
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
ax.yaxis.set_major_locator(ticker.MultipleLocator(5))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(30))
ax.set_yticks(np.arange(20, 100,5))
sns.barplot(x="SEASON", y='WINS_PERC', hue="TEAM", data=plot_data, ax=ax)

In [ ]:
wl_season_data = plot_data

### Three Points Field Goalds 

In [ ]:
dfi = raw_ds
dfi

In [ ]:
plot_data = dfi.groupby(by=["SEASON"]).mean()[["FGA", "FG3A"]].reset_index()
plot_data.SEASON.unique()

In [ ]:
import matplotlib.ticker as ticker
fig_dims = (10, 6)
fig, ax = plt.subplots(figsize=fig_dims, sharex=True, sharey=True)
ax.set(xticks=plot_data.SEASON.unique())
ax = sns.lineplot(x=plot_data.SEASON.unique(), y=plot_data.FG3A.values, ax=ax, marker="o")
ax.set_title("Promedio por partido de intento de tiros de 3PTS")
ax.set(xlabel="Temporadas", ylabel = "Intentos de tiro de 3PTS")
plt.show()

In [ ]:
winner_by_3pta = dataset.reset_index()
winner_by_3pta['WINNER_3PTA'] = 0
cond = ((winner_by_3pta.FG3A_HOME > winner_by_3pta.FG3A_AWAY) & (winner_by_3pta.HOME_WINS == 1))
winner_by_3pta.loc[cond, 'WINNER_3PTA'] = 1
cond = ((winner_by_3pta.FG3A_HOME < winner_by_3pta.FG3A_AWAY) & (winner_by_3pta.HOME_WINS == 0))
winner_by_3pta.loc[cond, 'WINNER_3PTA'] = 1

In [ ]:
winner_by_3pta.groupby(by=["SEASON", 'WINNER_3PTA']).count()[['GAME_ID']]

In [ ]:
grouped = dfi[['SEASON', 'WL_HOME', 'GAME_ID']].groupby(by=['SEASON', 'WL_HOME']).count()
grouped = grouped.reset_index(level=0).reset_index(level=0)
grouped_total = grouped[['SEASON', 'GAME_ID']].groupby(by=['SEASON']).sum()
grouped_total = grouped_total.reset_index(level=0)
plot_data = pd.merge(grouped, grouped_total, suffixes=['', '_S'], on=['SEASON', 'SEASON'])
plot_data.rename(columns={"GAME_ID": "W_COUNT", "GAME_ID_S": "GAMES_COUNT"}, inplace=True)
plot_data["TEAM"] = np.where(plot_data['WL_HOME'] == 'W', 'HOME', 'AWAY')
plot_data["WINS_PERC"] = (plot_data["W_COUNT"] / plot_data["GAMES_COUNT"]) * 100
plot_data.head()

In [ ]:
import matplotlib.ticker as mtick
import matplotlib.ticker as ticker
fig_dims = (20, 10)
fig, ax = plt.subplots(figsize=fig_dims, sharex=True, sharey=True)
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
ax.yaxis.set_major_locator(ticker.MultipleLocator(5))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(30))
ax.set_yticks(np.arange(20, 100,5))
sns.barplot(x="SEASON", y='FG3A', hue="LOCATION", data=plot_data, ax=ax)

### How much have offensive NBA stats changed over time?

In [ ]:
plot_data = dfi.groupby(by=["SEASON"]).mean()[['FGA', 'REB', "AST", 'STL', 'PTS']].reset_index()

In [ ]:
import matplotlib.ticker as ticker
fig_dims = (10, 6)
fig, ax = plt.subplots(figsize=fig_dims, sharex=True, sharey=True)
ax.set(xticks=plot_data.SEASON.unique())
ax = sns.lineplot(data=pd.melt(plot_data, ['SEASON']), x='SEASON', y='value', hue='variable', ax=ax, marker="o")
ax.set_title("Promedio por partido de estadisticas ofensivas")
ax.set(xlabel="Temporadas", ylabel = "Promedio")
plt.show()

## Features  correlations

### Utils functions

In [32]:
def plot_heatmap(data, title):
    mask = np.zeros_like(pearson_corr)
    mask[np.triu_indices_from(mask)] = True
    with sns.axes_style("white"):
        f, ax = plt.subplots(figsize = (30, 24))
        heatmap = sns.heatmap(data, mask=mask, square=True,
                        linewidth=1,
                        annot=True,
                        annot_kws={'size' : 10} 
                        )
#     plt.figure(figsize = (30, 24))
#     heatmap = sns.heatmap(data,
#             linewidth=1,
#             annot=True,
#             annot_kws={'size' : 10} )
    plt.title(title, fontsize=25)
    plt.show()
    return heatmap

def save_heatmap(heatmap, name):
    figure = heatmap.get_figure()    
    figure.savefig(f'plots/{name}.png')
    
def top_corr_features(corr):
    c = corr.abs()
    sol = (c.where(np.triu(np.ones(c.shape), k=1).astype(np.bool))
                 .stack()
                 .sort_values(ascending=False))
    return pd.DataFrame(sol)

### Generate correlation matrix

In [ ]:
pearson_corr = df.corr(method='pearson')
spearman_corr = df.corr(method='spearman')
kendall_corr = df.corr(method='kendall')

### Pearson Correlations

In [ ]:
mask = np.zeros_like(pearson_corr)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize = (30, 24))
    ax = sns.heatmap(pearson_corr, mask=mask, square=True,
                    linewidth=1,
                    annot=True,
                    annot_kws={'size' : 10} 
                    )
save_heatmap(plot_heatmap(pearson_corr, "Pearson Correlation"), "pearson_corr")

In [ ]:
top_corr_features_df = top_corr_features(pearson_corr)
qgrid_widget = qgrid.show_grid(top_corr_features_df, show_toolbar=True , grid_options={'forceFitColumns': False, 'defaultColumnWidth': 200})
qgrid_widget

Se observa que existe muchos features altamente correlacionados, por lo que se evaluara que tan insprecindible son y como es el comportamiento de los experimentos al eliminar estos.

### Spearman Correlation

In [ ]:
save_heatmap(plot_heatmap(spearman_corr, "Spearman Correlation"), "spearman_corr")

In [ ]:
top_corr_features_df = top_corr_features(spearman_corr)
qgrid_widget = qgrid.show_grid(top_corr_features_df, show_toolbar=True , grid_options={'forceFitColumns': False, 'defaultColumnWidth': 200})
qgrid_widget

### __Kendall correlation__

In [ ]:
save_heatmap(plot_heatmap(kendall_corr, "Kendall Correlations"), "kendall_corr")

In [ ]:
top_corr_features_df = top_corr_features(kendall_corr)
qgrid_widget = qgrid.show_grid(top_corr_features_df, show_toolbar=True , grid_options={'forceFitColumns': False, 'defaultColumnWidth': 200})
qgrid_widget

## Generate Panda Profiling Report

In [34]:
cols = config.X_EDA_COLS
report_df = df[df.SEASON >= 2013]
report_df = df[cols]
report_df = report_df.dropna()

In [35]:

profile = ProfileReport(report_df, title='Pandas Profiling Report', pool_size=4,
                        minimal=True,
                        explorative=True,
                           correlations={
             "pearson": {"calculate": True},
             "spearman": {"calculate": True},
             "kendall": {"calculate": True},
             "phi_k": {"calculate": False},
             "cramers": {"calculate": False},
         })

In [36]:
profile.to_file("plots/season_games_profile_report.html")

Para visualizar el reporte generado puede acceder a este link:

[Panda Profiling Report](plots/season_games_profile_report.html)

## Generate sweetviz report

In [37]:
#analyzing the dataset
report = sv.analyze(report_df, pairwise_analysis='on')
#display the report
report.show_html('plots/sweetviz_report.html')

:FEATURES DONE:                    |█████████████████████| [100%]   00:36  -> (00:00 left)
:PAIRWISE DONE:                    |█████████████████████| [100%]   00:04  -> (00:00 left)


Creating Associations graph... DONE!
Report plots/sweetviz_report.html was generated! NOTEBOOK/COLAB USERS: no browser will pop up, the report is saved in your notebook/colab files.


Para visualizar el reporte generado puede acceder a este link:

[Sweetviz](plots/sweetviz_report.html)

## Visualization of correlation between dependent variable and features

In [ ]:
#target = df[config.y_columns]
target = df["HOME_TEAM_WINS"]
features = df.drop(config.y_columns, axis=1)

### Drop categorical values

In [ ]:
#features.drop(["HT_RANK", "HT_CLASS", "VT_RANK", "VT_CLASS"], axis=1, inplace=True)

In [ ]:
features_names = list(features.columns)

## Yellowbrick Visualizer

In [ ]:
plt.figure(figsize = (15, 15))
visualizer = FeatureCorrelation(labels=features_names)
visualizer.fit(features, target)
visualizer.poof()